# Functional Programming for pythonistas
Bianca Gibson

# Rampant side effects

# Shifting sands of mutable state beneath your feet

# Enter Functional Programming

<img src="assets/portrait.jpg" width="400">

# What should you get out of this?

# What we will cover

# Stop things changing behind my back

# Immutability

# story

# Code example
Use list comps as first example, then point out similarity to map

# What are side effects?

# What do they have to do with FP?

In [ ]:
#Code example

# Referential transparency

In [ ]:
# code example

# Teaching

# Learning more

# Questions

# Stop things changing behind our backs